In [4]:
#Выполнить предобработку данных (продумать самостоятельно)

#Поделить на две выборки - валидационную и обучающую.

#Предложить 5 принципиально разных архитектур (количесвто слоёв, число нейронов, тип слоя, функция активации и т.д.)

#Каждую архитектуру обучить, визуализировать кривые обучения (для тестовой и валидационной выборок). Сравнить между собой.

#Написать утилиту, которая будет строить график предсказания будущих значений на основе начального фрагмента.

#Написать классификатор, который выдаёт рекомендации по конвертации актива.

#Adjust data (choose adjustment by yourself)

#Split into 2 datasets for validation and learning

#Suggest 5 different architectures (number of layers, number of neurons, layer types, activation function etc.)

#Create an util for graphic represetation of learning curves (for test and validation sets). Compare these graphs

#Create a classificator for active conversion recommendations

In [5]:
import matplotlib.pyplot as plt

Let's read the data set and plot it

In [ ]:
data = eval(open('task1.txt', 'r').read())
print(data)